In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

In [ ]:
df_train = pd.read_csv('.csv')
df_test = pd.read_csv('.csv')

In [ ]:
df_train.head(5)